# MGMTMSA 408 -- Lecture 6: Transportation

In this notebook, we will be learning how to solve the traveling salesman problem (TSP), a well-known problem in operations research, using Gurobi.

The data set we will be using is a data set of Starbucks locations. 

This lecture will also serve as a vehicle to introduce *lazy constraints*, a computational paradigm implemented in modern solvers such as Gurobi for solving problems with extremely large numbers of constraints. Lazy constraint generation is of great utility in many other problems. 

_Note:_ This Jupyter Notebook uses a package for Python called Folium in order to plot some  maps. This package is not usually included with base installations of Python, so you may need to install it. If your Python is set up through Anaconda, you can open up a terminal and type 

`conda install -c conda-forge folium`

to install it. Alternatively, if you use pip, you can also install it by opening a terminal and typing

`pip install folium`


## Loading the Starbucks data

Let's go ahead and load the Starbucks data using Pandas and Numpy:

In [22]:
import pandas as pd
import numpy as np;

# Load the Starbucks locations:
starbucks_locations = pd.read_csv("starbucks_address_lat_lon.csv")

# Load the distance matrix:
starbucks_distance_mat = pd.read_csv("starbucks_dist_mat_full_noNAs.csv")

# Let's take a look at the first few rows of each:
print()
print(starbucks_locations.head())
print()
print(starbucks_distance_mat.head())


   Num                Location               Address..1              City  \
0    1      Olympic & Westwood      2215 Westwood Blvd.       Los Angeles   
1    2          Pico & Midvale   10911 W Pico Boulevard       Los Angeles   
2    3      Olympic & Sawtelle      11280 Olympic Blvd.       Los Angeles   
3    4     Westwood & Missouri      1898 Westwood Blvd.          Westwood   
4    5  Santa Monica & Pontius  11155 Santa Monica Blvd  West Los Angeles   

  State    ZIP        lat         lon  
0    CA  90064  34.043188 -118.431690  
1    CA  90064  34.039673 -118.430021  
2    CA  90064  34.037916 -118.440790  
3    CA  90025  34.048374 -118.435344  
4    CA  90025  34.047571 -118.445251  

   Time  Time_traffic  Distance  Status  OriginInd  DestinationInd
0   168           149       659     NaN          1               2
1   255           233      1225     NaN          1               3
2   174           146       700     NaN          1               4
3   368           286      

Let's visualize these locations in Folium:

In [23]:
import folium

# Figure out where the "center" of the plot should be
avg_lat = starbucks_locations['lat'].mean()
avg_lon = starbucks_locations['lon'].mean()

# Create the folium map
starbucksmap = folium.Map(location=[avg_lat, avg_lon], 
                  zoom_start = 13, detect_retina = True)
starbucksmap

Next, let's put markers down for the locations in our data set:

In [24]:
for i in range(starbucks_locations.shape[0]):
    folium.CircleMarker([starbucks_locations['lat'][i], starbucks_locations['lon'][i]], 
                        popup= 'Starbucks ' + str(i) + '- ' + starbucks_locations['Location'][i],
                        radius = 3,
                        color = 'blue',
                        fill = True,
                        fillcolor = 'blue').add_to(starbucksmap)
starbucksmap

We can see that most of the locations in West LA, with a few farther away in the Playa Vista and Inglewood areas. 

## An initial formulation of the TSP problem 

Let us start to formulate the TSP problem. The first formulation we will come up with is not the right one (we will see why), but it will get us on the right track.

The first step is to convert starbucks_distance_mat into something we can deal with. Before we do this, we should take note that starbucks_distance_mat contains several pieces of information:
- Time: this is the "average" travel time by car (as per the Google Maps API) between the OriginInd location and the DestinationInd location
- Time_traffic: this is the travel time by car taking into account "current" (= as of time of Google Maps API query; late evening) traffic conditions 
- Distance: distance in meters (by car)

For this demonstration, let's use the Time column to formulate the problem. Thus, the goal is to decide in what order to visit all the locations, so as to minimize the total driving time required. 

In [25]:
# We will proceed a little differently -- just to show you another way of representing the data. 

# First, let us "zip" the origin and destinations:
origin_dest_zipped = list(zip(starbucks_distance_mat['OriginInd'] - 1, starbucks_distance_mat['DestinationInd'] - 1))

# Let's see what happened
# print(origin_dest_zipped)

# This is an array of Python tuples. Each tuple corresponds to an origin-destination pair. 
# We'd like to use these tuples to define a dictionary. To do this, we can use zip() again
# together with dict():
travel_time_dict = dict( zip(origin_dest_zipped, starbucks_distance_mat['Time']) )
# In general, running dict( zip(a,b)) will create a dictionary where the keys are given by a,
# and the values are given by b. 

# Let's see this in action. Consider pair (28, 7):
print(starbucks_locations.loc[28])
print(starbucks_locations.loc[7])

# Let's see what the travel time is:
print("Travel time from 28 to 7: ", travel_time_dict[(28,7)])
# 861 seconds; roughly 15 minutes. 

# Notice that the times are not symmetric:
print("Travel time from 7 to 28: ", travel_time_dict[(7,28)])


Num                                 29
Location           Santa Monica & 26th
Address..1    2461-A Santa Monica Blvd
City                      Santa Monica
State                               CA
ZIP                              90404
lat                          34.032694
lon                         -118.47534
Name: 28, dtype: object
Num                              8
Location      Ralphs-Westwood #759
Address..1       10861 Weyburn Ave
City                      Westwood
State                           CA
ZIP                          90024
lat                      34.062651
lon                    -118.443942
Name: 7, dtype: object
Travel time from 28 to 7:  861
Travel time from 7 to 28:  826


Excellent -- we now have the travel_time_dict object set up. Let's now create our Gurobi model. 

To start with, let's actually just solve a small version of the problem with the first 4 locations. 

In [26]:
# Define the number of locations as 4:
nStarbucks = 4;

# Next: extract the od_pairs 
# filter(f,x) will apply the function f to each element of x, 
# and keep only those for which f evaluates to true.
def filter_fn( t ):
    flag = t[0] < nStarbucks and t[1] < nStarbucks
    return flag


print(filter_fn( (0,3)))

od_pairs = list(filter( filter_fn, travel_time_dict.keys())) #returns only elements that satisfy the condition

print(od_pairs)


# Next, let's build the Gurobi model:
from gurobipy import *

m = Model()

# Create the variables;
# We will do something different -- let's give m.addVars the od_pairs array
x = m.addVars(od_pairs, vtype = GRB.BINARY)

# Create the constraints:
for i in range(nStarbucks):
    m.addConstr( sum(x[i,j] for j in range(nStarbucks) if j != i ) == 1)
    m.addConstr( sum(x[j,i] for j in range(nStarbucks) if j != i ) == 1)

# Create the objective: 
m.setObjective(sum( travel_time_dict[i,j] * x[i,j] for (i,j) in od_pairs ), GRB.MINIMIZE)

# # Solve it:
m.update()
m.optimize()

# print( travel_time_dict[(32,54)])
# print( travel_time_dict[32,54])


True
[(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3), (1, 0), (2, 0), (2, 1), (3, 0), (3, 1), (3, 2)]
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 8 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x26fa552f
Variable types: 0 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 861.0000000
Presolve time: 0.01s
Presolved: 8 rows, 12 columns, 24 nonzeros
Variable types: 0 continuous, 12 integer (12 binary)

Root relaxation: objective 8.410000e+02, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0

Let's see what the solution looks like:

In [27]:
# Obtain the solution:
#od_pairs = filter( filter_fn, travel_time_dict.keys())
x_val = {}
for (i,j) in od_pairs:
    x_val[(i,j)] = x[i,j].x

print(x_val)


{(0, 1): 0.0, (0, 2): 0.0, (0, 3): 1.0, (1, 2): 1.0, (1, 3): -0.0, (2, 3): -0.0, (1, 0): 0.0, (2, 0): 0.0, (2, 1): 1.0, (3, 0): 1.0, (3, 1): -0.0, (3, 2): -0.0}


In [28]:
np.zeros(10).astype(int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

This is not so easy to parse -- let's instead extract only those x_{i,j}'s that are 1:

In [29]:
sequence = [ (i,j) for (i,j) in od_pairs if x[i,j].x > 0.5] #this is a robust way to check if they're 1 because there might be precision issues in gurobi 0 ~ 0
print("Sequence: ", sequence)

Sequence:  [(0, 3), (1, 2), (2, 1), (3, 0)]


So in words, we travel as follows:
- 2 -> 1
- 1 -> 2
- 3 -> 0
- 0 -> 3

What is the problem here? We do not have a cycle! We go from 2 to 1 and back to 2; then we go from 0 to 3 to 0. We want a continuous path that connects all the nodes.

The paths 2 -> 1 -> 2 and 0 -> 3 -> 0 have a name: they are called _subtours_. We will next see how we can eliminate these subtours.

## Eliminating the subtours. 

Let's add constraints to remove these subtours:

In [30]:
# Remove 2 -> 1 -> 2
m.addConstr(x[2,1] + x[1,2] <= 1)

# Remove 0 -> 3 -> 0
m.addConstr(x[0,3] + x[3,0] <= 1)

# Update + optimize:
m.update()
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10 rows, 12 columns and 28 nonzeros
Model fingerprint: 0xe638b6dc
Variable types: 0 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint R8 by 1.000000000

Found heuristic solution: objective 861.0000000
Presolve time: 0.00s
Presolved: 10 rows, 12 columns, 28 nonzeros
Variable types: 0 continuous, 12 integer (12 binary)

Root relaxation: cutoff, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | I

Let's extract the sequence again: 

In [31]:
# initiate sequence
sequence = [ (i,j) for (i,j) in od_pairs if x[i,j].x > 0.5]
print("Sequence: ", sequence)

Sequence:  [(0, 3), (1, 0), (2, 1), (3, 2)]


The sequence is now: 0 -> 3 -> 2 -> 1 -> 0 -- which is a bona fide tour!

## Solving the TSP generally

The logic we applied above can be generalized when we find subtours of more than two locations. We can consider enforcing all such constraints upfront, but we would have an extremely large number of constraints.

Instead, we will add these constraints (in the TSP literature, these are called *subtour elimination constraints*) in a lazy way. To do this, we will make use of lazy constraint generation.

To add these constraints using lazy constraint generation, we need two ingredients:
1. We need a function that will output all the subtours for us.
2. We need a function that Gurobi can invoke to check whether any subtours exist (using the function in (1)) and if so, add the necessary constraints.

Let's start with the first part. 

In [32]:
#testing next_node function
sequence
node = 3
#tests whether first element of each tuple in sequence == node 
next_node = list(filter(lambda t: t[0] == node, sequence))
print(next_node)
#extracts tuple from list
print(next_node[0])
#extracts end_point_node from tuple
print(next_node[0][1])

[(3, 2)]
(3, 2)
2


In [33]:
x = [1,2,3]
x.pop()
x

[1, 2]

In [34]:
# def getPair(t):
#     return (t[0] == node)

# def getPair_nextnode(t):
#     return (t[0] == next_node)

# we are trying to discover subtours
def getSubtours(sequence):
    subtour_list = []
    unvisited = list(range(nStarbucks))
    
    # iterate through unvisited list till its out of elements
    while ( len(unvisited) > 0 ):
        # pop removes last element from unvisited and stores it in node variable
        node = unvisited.pop()
        
        # initiate subtour list and append node to it
        subtour = []
        subtour.append(node)
        
        # filter sequence to get the next node
        next_node = list(filter(lambda t: t[0] == node, sequence))[0][1]
        
        # while next_node is in unvisited list, append it to subtour list and remove it from unvisited list
        # this loop identifies different subtours
        while (next_node in unvisited):
            # append next_node to subtour list
            subtour.append(next_node)
            # remove next_node from unvisited list
            unvisited.remove(next_node)
            # filter sequence to get the next node
            next_node = list(filter(lambda t: t[0] == next_node, sequence))[0][1]
            
        # append the discovered subtour to subtour_list
        subtour_list.append(subtour)
    
    return subtour_list

subtour_list = getSubtours(sequence)
print("subtour_list:")
print(subtour_list)
print()
print("sequence:")
print(sequence)

subtour_list:
[[3, 2, 1, 0]]

sequence:
[(0, 3), (1, 0), (2, 1), (3, 2)]


Arnav's Note: if subtour list only has one element, it means there is only one subtour of the sequence which mean it is the tour of all solutions and your solution is optimal.

__Exercise__: For the list `sequence` below, what are the sets of nodes that `getSubtours` will output? In what order will it visit the nodes?

In [35]:
sequence = [ (0,5), (1,7), (5,4), (7,9), (2,6), (4,2), (6,0), (9,3), (8,10), (3,1), (10,8)]
nStarbucks = 11

# 0, 5, 4, 2, 6, (0)
# 1, 7, 9, 3, (1)
# 8, 10, (8)

# 10, 8, (10)
# 9, 3, 1, 7, (9)
# 6, 0, 5, 4, 2, (6)

getSubtours(sequence)


[[10, 8], [9, 3, 1, 7], [6, 0, 5, 4, 2]]

__Exercise__: What does the size of `subtour_list` correspond to? 

__Exercise__: What does it mean if `len(subtour_list) == 1`? 

Now, we need to define the lazy constraint callback. 

In optimization solvers, a callback function is a function that gets called in certain places in the solution process. A lazy constraint callback is a callback function that will get called whenever the solver encounters a new integer solution. There are other types of callbacks (such as information callbacks, which can be used to query information about the solution process). 

In [36]:
#lazy constraint callback function helps you customize how Gurobi works, gets called sometimes
def eliminateSubtours(model, where):
    # MIPSOL is a flag indicating that we found a new integer solution
    if (where == GRB.Callback.MIPSOL):
        x_val = model.cbGetSolution(x)
        # same sequence as before
        sequence = [ (i,j) for (i,j) in od_pairs if x_val[i,j] > 0.5]
        subtour_list = getSubtours(sequence)
        # this is code equivalent of sigma(sigma(x[i,j])) <= len(subtour) - 1
        if (len(subtour_list) > 1):
            for subtour in subtour_list:
                # add a constraint to the model via model.cbLazy
                # add as you go, not all at once
                model.cbLazy( sum(x[i,j] for i in subtour for j in subtour if i != j) <= len(subtour) - 1)

Let's now try this out on the 4 location problem again:

In [37]:
nStarbucks = 4

m = Model()

x = m.addVars(od_pairs, vtype = GRB.BINARY)

for i in range(nStarbucks):
    m.addConstr( sum(x[i,j] for j in range(nStarbucks) if j != i ) == 1)
    m.addConstr( sum(x[j,i] for j in range(nStarbucks) if j != i ) == 1)

m.setObjective(sum( travel_time_dict[i,j] * x[i,j] for (i,j) in od_pairs ), GRB.MINIMIZE)

m.update()

# Enable lazy constraints
# customise how gurobi works using parameters
m.params.LazyConstraints = 1

# Supply the callback to Gurobi:
m.optimize(eliminateSubtours)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 8 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x26fa552f
Variable types: 0 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 861.0000000
Presolve time: 0.00s
Presolved: 8 rows, 12 columns, 24 nonzeros
Variable types: 0 continuous, 12 integer (12 binary)

Root relaxation: objective 8.410000e+02, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  841.00000    0    -  861.00000  841.00000  2.32%     -    0

This is the same objective value that we saw before. Let's see the solution.

In [38]:
# constructing the sequence using the new optimisation model using lazy constraint callback function
sequence = [ (i,j) for (i,j) in od_pairs if x[i,j].x > 0.5]
print("Sequence: ", sequence)

# Test out getSubtours with updated model:
subtour_list = getSubtours(sequence)
print("Subtours: ", subtour_list)

Sequence:  [(0, 3), (1, 0), (2, 1), (3, 2)]
Subtours:  [[3, 2, 1, 0]]


## Solving the full 61 Starbucks problem

Let's now try to solve the full problem, with all 61 locations. 

In [39]:
# Update the number of locations
nStarbucks = 61

# Don't forget to update the set of OD pairs:
od_pairs = travel_time_dict.keys()

# The rest is the same as before:
m = Model()

x = m.addVars(od_pairs, vtype = GRB.BINARY)

for i in range(nStarbucks):
    m.addConstr( sum(x[i,j] for j in range(nStarbucks) if j != i ) == 1)
    m.addConstr( sum(x[j,i] for j in range(nStarbucks) if j != i ) == 1)

m.setObjective(sum( travel_time_dict[i,j] * x[i,j] for (i,j) in od_pairs ), GRB.MINIMIZE)

m.update()

# Enable lazy constraints
m.params.LazyConstraints = 1

# Supply the callback to Gurobi:
m.optimize(eliminateSubtours)

Set parameter LazyConstraints to value 1


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads


User-callback calls 6, time in user-callback 0.00 sec


GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license

I hope you find this striking -- we just solved the full problem, with 61 locations, in less than a second!

The objective value is 16816 seconds (= 4 hours and 40 minutes). 

## Visualizing the solution

Let's now visualize the solution.

Let's get the sequence of Starbucks locations, in the order that they are visited:

In [21]:
sequence = [ (i,j) for (i,j) in od_pairs if x[i,j].x > 0.5]
subtour_list = getSubtours(sequence)
complete_tour = subtour_list[0]
complete_tour.append( complete_tour[0] )
print("Complete tour: ", complete_tour)

AttributeError: Unable to retrieve attribute 'x'

Let's add this tour to the Folium map:

In [25]:
tour_lat_lon = list(zip( starbucks_locations['lat'][complete_tour], starbucks_locations['lon'][complete_tour])) 

poly = folium.PolyLine( tour_lat_lon, color="red", weight=2.5, opacity=1)

poly.add_to(starbucksmap)

starbucksmap